In [7]:
import numpy as np
from models.nODE import nODE, make_nODE_from_parameters
import scipy.io

In [8]:
def distribute_parameters(param,adj):
    W1 = np.zeros((adj.shape[0],adj.shape[1]))
    k = 0
    for i in range(0, adj.shape[0]):
        for j in range(0, adj.shape[1]):
            if adj[i,j] > 0:
                W1[i,j] = param[k]
                k += 1

    gamma = np.zeros(adj.shape[0])
    for i in range(0, adj.shape[0]):
        gamma[i] = -param[k]
        k += 1

    b2 = np.zeros(adj.shape[0])
    for i in range(0, adj.shape[0]):
        b2[i] = param[k]
        k += 1
        
    b1 = np.zeros(adj.shape[0])
    for i in range(0, adj.shape[0]):
        b1[i] = param[k]
        k += 1

    W2 = np.zeros((adj.shape[0],adj.shape[1]))
    for i in range(0, adj.shape[0]):
        for j in range(0, adj.shape[1]):
            if i == j:
                W2[i,j] = param[k]
                k += 1
    
    return W1, gamma, b1, b2, W2


In [14]:
param1 = np.loadtxt('data/dyn_bifurcating/fit_cluster0_2_3.txt')
param2 = np.loadtxt('data/dyn_bifurcating/fit_cluster0.txt')

adj1 = scipy.io.loadmat('data/dyn_bifurcating/net.mat')['Aref']
adj1[2,3] = 0
adj2 = scipy.io.loadmat('data/dyn_bifurcating/net.mat')['Aref']

(W1_1, gamma_1, b1_1, b2_1, W2_1) = distribute_parameters(param1,adj1)
(W1_2, gamma_2, b1_2, b2_2, W2_2) = distribute_parameters(param2,adj2)

# make_nODE_from_parameters(Gamma, Win=None, bin=None, Wout=None, bout=None)
nODE1 = make_nODE_from_parameters(gamma_1, W1_1, b1_1, W2_1, b2_1)
nODE2_base = make_nODE_from_parameters(gamma_2, W1_2, b1_2, W2_2, b2_2)

In [15]:
from models.ranking import rank

rank(nODE1,nODE2_base)

[0.6984886635569303, 0.2959223985671997, 1.0]

In [16]:
param1 = np.loadtxt('data/dyn_bifurcating/fit_cluster0_2_3_improved.txt')
param2 = np.loadtxt('data/dyn_bifurcating/fit_cluster0.txt')

adj1 = scipy.io.loadmat('gae_results/dyn_bifurcating_2_3.mat')['inferred_adj']
adj1 = (adj1>0.5).astype(int)
adj2 = scipy.io.loadmat('data/dyn_bifurcating/net.mat')['Aref']

(W1_1, gamma_1, b1_1, b2_1, W2_1) = distribute_parameters(param1,adj1)
(W1_2, gamma_2, b1_2, b2_2, W2_2) = distribute_parameters(param2,adj2)

# make_nODE_from_parameters(Gamma, Win=None, bin=None, Wout=None, bout=None)
nODE1 = make_nODE_from_parameters(gamma_1, W1_1, b1_1, W2_1, b2_1)
nODE2_gae = make_nODE_from_parameters(gamma_2, W1_2, b1_2, W2_2, b2_2)

In [17]:
from models.ranking import rank

rank(nODE1,nODE2_gae)

[0.0, 0.2195059061050415, 0.07692307692307687]

In [21]:
nODE2_gae.adjacency_matrix()

tensor([[ 0.,  1.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  1.,  1.,  0.,  0.],
        [-1.,  0.,  0., -1., -1., -1., -1.],
        [-1.,  0.,  0.,  1.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [22]:
nODE2_base.adjacency_matrix()

tensor([[ 0.,  1.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  1.,  1.,  0.,  0.],
        [-1.,  0.,  0., -1., -1., -1., -1.],
        [-1.,  0.,  0.,  1.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [31]:
np.abs(nODE2_gae.adjacency_matrix()) - np.abs(nODE1.adjacency_matrix())

tensor([[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]])

0.30151135

3.3166249

1.0

In [23]:
true_network = np.abs(nODE1.adjacency_matrix())
found_network = np.abs(nODE2_gae.adjacency_matrix())
ranking_network = 1 - np.linalg.norm(true_network - found_network) / np.max([1., np.linalg.norm(true_network)])

In [24]:
ranking_network

-0.45296637217203783

In [25]:
np.linalg.norm(true_network - found_network)

4.358899

In [26]:
np.linalg.norm(true_network)

3.0

In [30]:
nODE1.adjacency_matrix()

tensor([[0., 0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 1., 0., 0.],
        [0., 0., 1., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]])

In [29]:
nODE2_gae.adjacency_matrix()

tensor([[ 0.,  1.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  1.,  1.,  0.,  0.],
        [-1.,  0.,  0., -1., -1., -1., -1.],
        [-1.,  0.,  0.,  1.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.]])